In [ ]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip

--2023-04-22 19:13:07--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.164.168
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.164.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895569552 (854M) [binary/octet-stream]
Saving to: ‘datasetb2d9982.zip’

datasetb2d9982.zip  100%[===================>] 854.08M  13.1MB/s    in 66s     

2023-04-22 19:14:14 (12.9 MB/s) - ‘datasetb2d9982.zip’ saved [895569552/895569552]



In [ ]:
!unzip /content/datasetb2d9982.zip -d data

Archive:  /content/datasetb2d9982.zip
   creating: data/dataset/
  inflating: data/dataset/sample_submission.csv  
  inflating: data/dataset/train.csv  
  inflating: data/dataset/test.csv   


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import csv
import os
import random
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
import sklearn.model_selection as model_selection
from transformers import BertForSequenceClassification, BertTokenizerFast
import time
import copy
from tqdm.notebook import tqdm

In [ ]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
set_seed()

In [ ]:
df=pd.read_csv("/content/data/dataset/test.csv")

In [ ]:
temp=df

In [ ]:
temp=temp.fillna(" ")

In [ ]:
temp.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,,,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",,,6318


## Preprocess the data by processing rows, concatenates certain elements from each row, removes duplicates, and constructs a list of sentences

In [ ]:
wholeSentence=[]
for idx,row in temp.iterrows(): 
    if(idx%100000==0):
        print(f"{idx} Done")
    wholeSentence.append(" ".join(list(set((row[1]+row[2]+" ".join(row[3].split(","))[:-1][1:]).split()))))

0 Done
100000 Done
200000 Done
300000 Done
400000 Done
500000 Done
600000 Done
700000 Done


In [ ]:
temp["WHOLE SENTENCE"]=wholeSentence

In [ ]:
temp=temp.reset_index(drop=True)

In [ ]:
dev=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
max_seq_len = 64

In [ ]:
class amazonDatasetTest(Dataset, tokenizer):
  def __init__(self,text,tokenizer):
    self.sentence=text
    self.tokenizer=tokenizer

  def __len__(self):
    return len(self.sentence)
  
  def __getitem__(self,idx):
    inp_tokens=self.tokenizer.encode_plus(self.sentence[idx], 
                                          padding="max_length", 
                                          add_special_tokens=True,
                                          max_length=max_seq_len, 
                                          truncation=True)
    inp_id=inp_tokens.input_ids
    inp_mask=inp_tokens.attention_mask

    return {
#         "text":self.sentence,
        "input_ids":torch.tensor(inp_id, dtype=torch.long),
        "input_attention_mask":torch.tensor(inp_mask, dtype=torch.long)
    }

In [ ]:
test_dataset = amazonDatasetTest(temp["WHOLE SENTENCE"], tokenizer)

In [ ]:
len(test_dataset)

734736

In [ ]:
test_dataloader=DataLoader(test_dataset,
                            batch_size=10,
                            shuffle=False,
                            num_workers=2)

In [ ]:
model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=16655)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id
model=torch.load("/content/drive/MyDrive/amazon ml challenge/Trained_model.pth")
print(model)
model.to(dev)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
predictions=[]
for data in tqdm(test_dataloader):
    input_ids = data["input_ids"].to(dev, dtype=torch.long)
    mask = data["input_attention_mask"].to(dev, dtype=torch.long)
    with torch.set_grad_enabled(False):
        outputs=model(
                    input_ids =input_ids,
                    attention_mask=mask
                )
        logits=outputs.logits
        
        _,preds=torch.max(logits,1)
        predictions+=list(preds.cpu().numpy())

  0%|          | 0/73474 [00:00<?, ?it/s]

In [ ]:
submission_df=pd.DataFrame(columns=['PRODUCT_ID','PRODUCT_LENGTH'])

In [ ]:
submission_df['PRODUCT_ID']=df['PRODUCT_ID']
submission_df['PRODUCT_LENGTH']=predictions

In [ ]:
submission_df['PRODUCT_LENGTH']=submission_df['PRODUCT_LENGTH'].map(id2lbl)

In [ ]:
submission_df.head()

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,13
1,1729783,10
2,1871949,10
3,1107571,10
4,624253,13


In [ ]:
submission_df.to_csv('PredictionFile.csv', index=None)